In [19]:
## Inicialização dos modelos
using DataFrames
using CSV
using Dates
ENV["lines"]=200

200

In [20]:
function chercher(DF,palavra)
    #funcao que para dado um df coluna, retorna a posicao das palavras escolhidas
    position=[]
    for i in range(1,size(DF)[1],step=1)
        if DF[i] == palavra
            append!( position, i )
        end
    end
    return position
end


function match(array1,array2)
    #Funcao que dado dois arrays de posicoes (vindos do chercher), retorna as posiçoes que sao comuns aos dois arrays
    multiple=[]
    for i in range(1,size(array1)[1],step=1)
        for j in range(1,size(array2)[1],step=1)
            if array1[i]==array2[j]
                append!( multiple, array1[i] )
            end
        end
    end
    return multiple
end

match (generic function with 1 method)

In [21]:
study_path = "D:/repositorio/auto-organon/testes/"; #raiz onde ficará todos os casos
ano = "2028"
lines = DataFrame(CSV.File(study_path*ano*"Lines.csv",header=2,delim=";"));
trafo = DataFrame(CSV.File(study_path*ano*"Transfo.csv",header=2,delim=";"));


In [22]:
#Defini-se aqui as barras as quais buscaremos as vizinhanças e também as Areas de interesse do estudo (ideal por todas as areas que temos elementos até a 3 viz)
bus = ["GILBU2-PI500"]
area = [701, 711, 761, 772, 773, 841, 881, 883]
#bus = ["PECEM2-CE230"]

8-element Vector{Int64}:
 701
 711
 761
 772
 773
 841
 881
 883

In [23]:
#Criação de headers para o dataframe das vizinhanças
nome=["From#","From Name","To# - Circ#","To Name","Area"] 
viz1=DataFrame()
viz2=DataFrame()
viz3=DataFrame()
for i in range(1,length(nome),step=1)
    if i == 1
        viz1[!,nome[i]]=Int[]
        viz2[!,nome[i]]=Int[]
        viz3[!,nome[i]]=Int[]
    else
        viz1[!,nome[i]]=String[]
        viz2[!,nome[i]]=String[]
        viz3[!,nome[i]]=String[]
    end
end

# Funções gerais de contigência para qualquer vizinhança

In [24]:
function cont(Surface,Bus,saida)
    #Função que busca as linhas e transformadores na 1 vizinhança das barras fornecidas, na area desejada.
    #Surface = Lista de áreas de interesse
    #Bus = Lista de barras que buscaremos as vizinhanças diretas
    #Saida = DataFrame onde será salvo os dados
    
    
    LinhasArea=[]
    TrafoArea=[]
    for a in Surface #Para filtrar o arquivo de linhas e transfo pela aréa igual a buscada
        LinhasArea=vcat(LinhasArea,chercher(lines[!,"Area"],a))
        TrafoArea=vcat(TrafoArea,chercher(trafo[!,"Area"],a))
    end
    #####
    
    
    elemL=[]
    elemT=[]
    for b in Bus #para determinar quais linhas possuem a barra saindo ou chegando com os nomes desejados
        elemL=vcat(elemL,chercher(lines[!,"From Name"],b))
        elemL=vcat(elemL,chercher(lines[!,"To Name"],b))

        elemT=vcat(elemT,chercher(trafo[!,"From Name"],b))
        elemT=vcat(elemT,chercher(trafo[!,"To Name"],b))
    end
    ####
    #para fazer o match cruzado entre as que estão na area desejada e as que possuem o nome de partida/chegada desejada
    Linhas = match(elemL,LinhasArea)
    Trafo =match(elemT,TrafoArea)
    
    for i in Linhas #adicionar no arquivo de saida as contingencias
        col1=lines[i,1]
        col2=lines[i,2]
        col3=lines[i,3]
        col4=lines[i,4]
        col5=string(lines[i,5])
        push!(saida,[col1,col2,col3,col4,col5])
    end

    for i in Trafo
        col1=trafo[i,1]
        col2=trafo[i,2]
        col3=trafo[i,3]
        col4=trafo[i,4]
        col5=string(trafo[i,5])
        push!(saida,[col1,col2,col3,col4,col5])
    end
    
    return saida
    
end

cont (generic function with 1 method)

In [25]:
function outOrganon(vizinhanca)
    #Função que retorna em formato texto o arquivo de contingencia pronto para ser posto no organon. Dessa vez exclui-se circuitos duplos ou mais
    #Vizinhanca = dataframe produzido pela função cont
    
    for i in range(1,size(vizinhanca)[1],step=1)
        partida=string(vizinhanca[i,1])
        indc=findfirst(isequal(' '), vizinhanca[i,3])
        chegada=vizinhanca[i,3][1:indc-1]
        
        indcir=findfirst(isequal('#'), vizinhanca[i,3])
        circ=vizinhanca[i,3][indcir+2:end]
        ncirc=parse(Int,circ)
        if ncirc>1 #pular segundos circuitos
            continue
        end
        
        
        
    
        namefrom=vizinhanca[i,2]
        nameto=vizinhanca[i,4]
        

        println("'"*namefrom*"_"*nameto*"\t'")
        println("BRANCH\t"*partida*"\t"*chegada*"\t01")
        println("\tEND /")
    end
     println("END /")
end

outOrganon (generic function with 1 method)

# Contingência 1 vizinhança

In [26]:
cont(area,bus,viz1)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,6075,BURITI-BA500,7190 # 06,GILBU2-PI500,701
2,7200,MIRACE-TO500,7190 # 03,GILBU2-PI500,881


In [27]:
outOrganon(viz1)

END /


# Contingência 2 vizinhança

In [28]:
cont2=[]
for i in range(1,size(viz1)[1],step=1)
    if viz1[i,2] in bus
        name=viz1[i,4]
    else 
        name=viz1[i,2]
    end
    cont2=vcat(cont2,name)
end
cont2=unique(cont2);

In [29]:
cont(area,cont2,viz2)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,6075,BURITI-BA500,5325 # 02,QUEIMA-PI500,701
2,6075,BURITI-BA500,5325 # 03,QUEIMA-PI500,701
3,6075,BURITI-BA500,7190 # 06,GILBU2-PI500,701
4,6075,BURITI-BA500,9478 # 01,ALTITU-BA500,701
5,6075,BURITI-BA500,44946 # 01,BARRA2-BA500,701
6,6060,GENDO2-BA500,6075 # 07,BURITI-BA500,701
7,6360,BARREI-BA500,6075 # 06,BURITI-BA500,701
8,7200,MIRACE-TO500,7190 # 03,GILBU2-PI500,881
9,7200,MIRACE-TO500,7204 # 02,LAJEAD-TO500,881


In [30]:
outOrganon(viz2)

'BURITI-BA500_ALTITU-BA500	'
BRANCH	6075	9478	01
	END /
'BURITI-BA500_BARRA2-BA500	'
BRANCH	6075	44946	01
	END /
'MIRACE-TO500_CO-MI1CAP500	'
BRANCH	7200	7301	01
	END /
'MIRACE-TO500_MI-GR1CAP500	'
BRANCH	7200	7201	01
	END /
'SPELAD-PA500_MIRACE-TO500	'


BRANCH	6754	7200	01
	END /
'LAJEAD-TO500_MIRACE-TO500	'
BRANCH	7204	7200	01
	END /
'MIRACE-TO500_MIRACE-TO000	'
BRANCH	7200	7208	01
	END /
END /


#  Contingência 3 vizinhança

In [31]:
cont3=[]
for i in range(1,size(viz2)[1],step=1)
    if viz2[i,2] in cont2
        name=viz2[i,4]
    else 
        name=viz2[i,2]
    end
    cont3=vcat(cont3,name)
end
cont3=unique(cont3);

In [32]:
cont(area,cont3,viz3)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,5380,MLG2---CE500,5325 # 03,QUEIMA-PI500,761
2,6075,BURITI-BA500,5325 # 02,QUEIMA-PI500,701
3,6075,BURITI-BA500,5325 # 03,QUEIMA-PI500,701
4,6075,BURITI-BA500,7190 # 06,GILBU2-PI500,701
5,7200,MIRACE-TO500,7190 # 03,GILBU2-PI500,881
6,6075,BURITI-BA500,9478 # 01,ALTITU-BA500,701
7,44946,BARRA2-BA500,44947 # 01,CORREN-BA500,701
8,6075,BURITI-BA500,44946 # 01,BARRA2-BA500,701
9,44945,CFORM2-BA500,44946 # 01,BARRA2-BA500,701


In [33]:
outOrganon(viz3)

'BURITI-BA500_ALTITU-BA500	'
BRANCH	6075	9478	01
	END /
'BARRA2-BA500_CORREN-BA500	'
BRANCH	44946	44947	01


	END /
'BURITI-BA500_BARRA2-BA500	'
BRANCH	6075	44946	01


	END /
'CFORM2-BA500_BARRA2-BA500	'
BRANCH	44945	44946	01
	END /
'GENDO2-BA500_OUROLN-BA500	'
BRANCH	6060	6070	01
	END /
'BARREI-BA500_CELEO--BA500	'
BRANCH	6360	44596	01
	END /
'BARREI-BA500_SERSOL-BA500	'
BRANCH	6360	44601	01
	END /


'NEWER1-BA500_BARREI-BA500	'
BRANCH	44700	6360	01
	END /
'EDNBAR-BA500_BARREI-BA500	'
BRANCH	44978	6360	01
	END /
'LAJEAD-TO500_MIRACE-TO500	'
BRANCH	7204	7200	01
	END /
'MIRACE-TO500_CO-MI1CAP500	'
BRANCH	7200	7301	01
	END /
'COLINA-TO500_CO-MI1CAP500	'
BRANCH	7300	7301	01
	END /
'GR-MI1CAP500_MI-GR1CAP500	'
BRANCH	7102	7201	01


	END /
'MIRACE-TO500_MI-GR1CAP500	'
BRANCH	7200	7201	01
	END /
'SPELAD-PA500_ITACAI-PA500	'
BRANCH	6754	6440	01
	END /
'SPELAD-PA500_INTEGR-PA500	'
BRANCH	6754	6560	01
	END /
'SPELAD-PA500_MIRACE-TO500	'
BRANCH	6754	7200	01
	END /
'XINGU--PA500_SPELAD-PA500	'
BRANCH	8100	6754	01
	END /
'ALTITU-BA500_ALTITU-BA230	'
BRANCH	9478	9479	01
	END /
'BARRA2SIN013_BARRA2-BA500	'
BRANCH	44948	44946	01


	END /
'GENDO2-BA500_GOURO-CER030	'
BRANCH	6060	9528	01
	END /
'GENDO2-BA500_GOR-T1-BA000	'
BRANCH	6060	42392	01
	END /
'BARREI-BA500_BRD-T1-BA000	'
BRANCH	6360	42386	01
	END /
'LAJEAD-TO500_LAJ-T1-TO000	'
BRANCH	7204	44031	01
	END /
'SPELAD-PA500_SPLAT1-PA000	'
BRANCH	6754	7261	01
	END /
'MIRACE-TO500_MIRACE-TO000	'
BRANCH	7200	7208	01
	END /
'MIRACEPCH138_MIRACE-TO000	'
BRANCH	7202	7208	01
	END /
'MIRACE-TO013_MIRACE-TO000	'
BRANCH	7205	7208	01
	END /
END /


# Contingências totais

In [34]:
contingencias=vcat(viz1,viz2)
#contingencias=vcat(contingencias,viz3) #Caso deseja-se incluir 3 vizinhança. porém para a 3 melhor fazer na mão
sort!(contingencias,[1,3])

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,6060,GENDO2-BA500,6075 # 07,BURITI-BA500,701
2,6075,BURITI-BA500,44946 # 01,BARRA2-BA500,701
3,6075,BURITI-BA500,5325 # 02,QUEIMA-PI500,701
4,6075,BURITI-BA500,5325 # 03,QUEIMA-PI500,701
5,6075,BURITI-BA500,7190 # 06,GILBU2-PI500,701
6,6075,BURITI-BA500,7190 # 06,GILBU2-PI500,701
7,6075,BURITI-BA500,9478 # 01,ALTITU-BA500,701
8,6360,BARREI-BA500,6075 # 06,BURITI-BA500,701
9,6754,SPELAD-PA500,7200 # 01,MIRACE-TO500,841


In [35]:
#Remoção de elementos que se repetem. ex: se temos 2 buses de interesse, a msm linha é primeira vizinhança dos 2 buses
sum=0
for i in findall(nonunique(contingencias))
    delete!(contingencias,i-sum)
    sum+=1
end

In [36]:
outOrganon(contingencias)

'BURITI-BA500_BARRA2-BA500	'
BRANCH	6075	44946	01
	END /
'BURITI-BA500_ALTITU-BA500	'
BRANCH	6075	9478	01
	END /
'SPELAD-PA500_MIRACE-TO500	'
BRANCH	6754	7200	01
	END /
'MIRACE-TO500_MI-GR1CAP500	'
BRANCH	7200	7201	01
	END /
'MIRACE-TO500_MIRACE-TO000	'
BRANCH	7200	7208	01
	END /
'MIRACE-TO500_CO-MI1CAP500	'
BRANCH	7200	7301	01
	END /
'LAJEAD-TO500_MIRACE-TO500	'
BRANCH	7204	7200	01
	END /


END /
